In [9]:
# ✅ Verificación de versión de Python
import sys

required: tuple[int, int, int] = (3, 10, 0)
current: tuple[int, int, int] = sys.version_info[:3]

assert current >= required, (
    f"⚠️  Se requiere Python {required[0]}.{required[1]}.{required[2]} o superior "
    f"para usar sintaxis moderna de tipos (str | None). "
    f"Detectado: {current[0]}.{current[1]}.{current[2]}"
)

print(f"✅ Python {current[0]}.{current[1]}.{current[2]} >= {required[0]}.{required[1]}.{required[2]}")
print("✅ Sintaxis moderna de tipos disponible (PEP 604: Union | None)")


✅ Python 3.13.3 >= 3.10.0
✅ Sintaxis moderna de tipos disponible (PEP 604: Union | None)


# 🧠 IA + Oracle: Hola mundo con Gemini API Key

Este notebook muestra cómo conectar una base de datos Oracle y generar consultas SQL desde lenguaje natural usando el modelo Gemini de Google vía su librería oficial `google.generativeai`.

🔑 Solo necesitas una clave API (`GOOGLE_API_KEY`).  
❌ No se requiere `project_id`, `region` ni cuenta de servicio.

Ideal para prototipos rápidos y prácticas educativas.

## 🔍 Comparativa: Vertex AI vs Gemini API Key

| Característica              | Vertex AI (AISuite)         | Gemini API Key (`google.generativeai`) |
|----------------------------|-----------------------------|----------------------------------------|
| Requiere `project_id`      | ✅ Sí                        | ❌ No                                   |
| Requiere `region`          | ✅ Sí                        | ❌ No                                   |
| Requiere cuenta de servicio| ✅ Sí                        | ❌ No                                   |
| Ideal para producción      | ✅                           | 🔸 Solo para prototipos y pruebas       |
| Simplicidad de uso         | 🔸 Más compleja              | ✅ Muy sencilla                         |

## 📚 Sobre las Buenas Prácticas de Código

Este notebook aplica **código profesional Python 3.10+** con:
- ✅ Type hints modernos (`str | None` en lugar de `Optional[str]`)
- ✅ Funciones limpias con UN solo `return` y guard clauses
- ✅ Manejo robusto de errores con excepciones específicas
- ✅ Documentación completa (docstrings con Args, Returns, Raises)

**📖 Para detalles completos sobre buenas prácticas de Python (con ejemplos ❌ MAL / ✅ BIEN), consulta:**
👉 **[`buenaspracticas.ipynb`](buenaspracticas.ipynb)**

**💡 Objetivo de este notebook:** Demostrar uso simplificado de Gemini API Key para generar SQL desde lenguaje natural.


In [10]:
# 🔐 Configurar Gemini con API Key
from dotenv import load_dotenv
import os
import google.generativeai as genai

def cargar_configuracion_gemini() -> tuple[str, genai.GenerativeModel]:
    """Carga configuración y crea modelo Gemini.
    
    Returns:
        Tupla con (API_KEY, modelo_gemini)
        
    Raises:
        ValueError: Si GOOGLE_API_KEY no está definida
    """
    load_dotenv()
    api_key: str | None = os.getenv("GOOGLE_API_KEY")
    
    # Guard clause
    if not api_key:
        raise ValueError("GOOGLE_API_KEY no está definida en .env")
    
    # Configurar y crear modelo
    genai.configure(api_key=api_key)
    model: genai.GenerativeModel = genai.GenerativeModel("gemini-2.0-flash-001")
    
    print("✅ Gemini configurado con API Key")
    return api_key, model

# Configurar Gemini
GOOGLE_API_KEY: str
model: genai.GenerativeModel
GOOGLE_API_KEY, model = cargar_configuracion_gemini()


✅ Gemini configurado con API Key


In [11]:
# 🔗 Conexión a Oracle con Type Hints
import oracledb

def obtener_config_oracle() -> dict[str, str | None]:
    """Obtiene configuración de Oracle desde variables de entorno.
    
    Returns:
        Diccionario con parámetros de conexión
    """
    config: dict[str, str | None] = {
        "ORACLE_HOST": os.getenv("ORACLE_HOST"),
        "ORACLE_PORT": os.getenv("ORACLE_PORT"),
        "ORACLE_SID": os.getenv("ORACLE_SID"),
        "ORACLE_USER": os.getenv("ORACLE_USER"),
        "ORACLE_PASSWORD": os.getenv("ORACLE_PASSWORD"),
    }
    return config


def conectar_oracle(config: dict[str, str | None]) -> tuple[oracledb.Connection, oracledb.Cursor]:
    """Establece conexión a Oracle y retorna conexión y cursor.
    
    Args:
        config: Diccionario con configuración de Oracle
        
    Returns:
        Tupla con (conexión, cursor)
        
    Raises:
        ValueError: Si faltan parámetros de configuración
        oracledb.DatabaseError: Si falla la conexión
    """
    # Validar configuración
    host: str | None = config.get("ORACLE_HOST")
    port: str | None = config.get("ORACLE_PORT")
    sid: str | None = config.get("ORACLE_SID")
    user: str | None = config.get("ORACLE_USER")
    password: str | None = config.get("ORACLE_PASSWORD")
    
    if not all([host, port, sid, user, password]):
        raise ValueError("Faltan parámetros de conexión a Oracle")
    
    # Crear conexión
    dsn: str = oracledb.makedsn(host, port, sid=sid)  # type: ignore
    connection: oracledb.Connection = oracledb.connect(
        user=user,  # type: ignore
        password=password,  # type: ignore
        dsn=dsn
    )
    cursor: oracledb.Cursor = connection.cursor()
    
    # Verificar conexión
    cursor.execute("SELECT USER FROM dual")
    usuario_conectado: str = cursor.fetchone()[0]  # type: ignore
    print(f"✅ Conectado a Oracle como: {usuario_conectado}")
    
    return connection, cursor

# Conectar a Oracle
config_oracle: dict[str, str | None] = obtener_config_oracle()
connection: oracledb.Connection
cursor: oracledb.Cursor
connection, cursor = conectar_oracle(config_oracle)


✅ Conectado a Oracle como: UBD2640


In [14]:
# 🗂️ Descubrimiento del esquema de BD

def obtener_esquema_bd(cursor: oracledb.Cursor, usuario: str | None) -> str:
    """Obtiene el esquema completo de la base de datos.
    
    Args:
        cursor: Cursor activo de Oracle
        usuario: Nombre del usuario/owner de las tablas
        
    Returns:
        String formateado con el esquema legible para LLM
        
    Raises:
        ValueError: Si el usuario es None
        oracledb.DatabaseError: Si falla la consulta
    """
    # Guard clause
    if not usuario:
        raise ValueError("Usuario de BD no definido")
    
    # Consultar esquema
    cursor.execute(f"""
        SELECT table_name, column_name
        FROM all_tab_columns
        WHERE owner = UPPER('{usuario}')
        ORDER BY table_name, column_id
    """)
    
    # Construir diccionario
    esquema_dict: dict[str, list[str]] = {}
    for table, column in cursor:
        esquema_dict.setdefault(table, []).append(column)
    
    # Formatear para humanos y LLM
    print("✅ Esquema de la base de datos:")
    for tabla, columnas in esquema_dict.items():
        print(f"  {tabla}({', '.join(columnas)})")
    
    # Formatear para LLM (más conciso)
    esquema_texto: str = "\n".join(
        f"{tabla}({', '.join(columnas)})" 
        for tabla, columnas in esquema_dict.items()
    )
    
    return esquema_texto

# Obtener esquema
esquema_texto: str = obtener_esquema_bd(cursor, config_oracle.get("ORACLE_USER"))


✅ Esquema de la base de datos:
  APLICACIONES(NOMBRE_APLIC, COMPANIA, PRECIO, WEB_APLICACION)
  CLIENTES(ID_CLIENTE, NOMBRE, EMAIL, CIUDAD)
  COMPATIBLE(NOMBRE_DOCUMENTO, DIRECTORIO, NOMBRE_APLICACION)
  DOC_TEXTO(ASCII, NUM_LINEAS, NUM_CARACTERES, TIPO, NOMBRE, DIRECTORIO)
  DOCUMENTO(NOMBRE, DIRECTORIO, FECHA_CREACION, COMENTARIO, TAMANO)
  PAGINA(NOMBRE, DIRECTORIO, NUMERO_PAGINA)
  PEDIDOS(ID_PEDIDO, ID_CLIENTE, ID_PRODUCTO, FECHA_PEDIDO, CANTIDAD)
  PRODUCTOS(ID_PRODUCTO, NOMBRE, PRECIO, CATEGORIA)


In [15]:
# 🧠 Pregunta en lenguaje natural
pregunta: str = "¿Qué productos compró Ana Gamez y cuánto costaron?"

print(f"📝 Pregunta del usuario: {pregunta}")


📝 Pregunta del usuario: ¿Qué productos compró Ana Gamez y cuánto costaron?


In [16]:
# 🤖 Generación de SQL con Gemini

def generar_sql_con_gemini(
    model: genai.GenerativeModel, 
    pregunta: str, 
    esquema: str
) -> str:
    """Genera consulta SQL desde lenguaje natural usando Gemini.
    
    Args:
        model: Modelo de Gemini configurado
        pregunta: Consulta del usuario en lenguaje natural
        esquema: Esquema de la BD en formato string
        
    Returns:
        Consulta SQL limpia y lista para ejecutar
    """
    prompt: str = f"""
Eres un asistente experto en SQL para Oracle. Genera solo la consulta SQL compatible con Oracle.
Usa este esquema de base de datos:
{esquema}

Pregunta del usuario:
{pregunta}
"""
    
    # Generar SQL
    response = model.generate_content(prompt)
    raw_sql: str = response.text.strip()
    
    # Limpiar SQL
    sql_limpio: str = _limpiar_sql_gemini(raw_sql)
    
    print("✅ SQL generado por Gemini:")
    print(sql_limpio)
    
    return sql_limpio


def _limpiar_sql_gemini(raw_sql: str) -> str:
    """Función auxiliar: Limpia el SQL generado por Gemini.
    
    Args:
        raw_sql: SQL crudo que puede contener Markdown
        
    Returns:
        SQL limpio en una línea, sin punto y coma
    """
    cleaned: str = raw_sql.strip()
    
    # Eliminar bloques Markdown
    cleaned = cleaned.strip("```sql").strip("```").strip()
    
    # Normalizar
    cleaned = cleaned.replace(";", "").replace("\n", " ").replace("\t", " ")
    
    # Eliminar espacios múltiples
    return " ".join(cleaned.split())

# Generar SQL
sql_generado: str = generar_sql_con_gemini(model, pregunta, esquema_texto)


✅ SQL generado por Gemini:
SELECT p.Nombre AS Nombre_Producto, p.Precio AS Precio_Producto FROM CLIENTES c JOIN PEDIDOS o ON c.ID_CLIENTE = o.ID_CLIENTE JOIN PRODUCTOS p ON o.ID_PRODUCTO = p.ID_PRODUCTO WHERE c.Nombre = 'Ana Gamez'


In [17]:
# 🧪 Ejecutar SQL con manejo de errores robusto
import pandas as pd

def ejecutar_sql_seguro(cursor: oracledb.Cursor, sql: str) -> tuple[bool, list[tuple] | None, str | None]:
    """Ejecuta SQL de forma segura y retorna estado.
    
    Args:
        cursor: Cursor activo de Oracle
        sql: Consulta SQL a ejecutar
        
    Returns:
        Tupla con (éxito, resultados, mensaje_error)
        - Si éxito=True: resultados contiene las tuplas, mensaje_error es None
        - Si éxito=False: resultados es None, mensaje_error contiene el error
    """
    # Guard clause
    if not sql.strip():
        return False, None, "La consulta SQL está vacía"
    
    try:
        cursor.execute(sql)
        resultados: list[tuple] = cursor.fetchall()
        print(f"✅ Consulta ejecutada: {len(resultados)} filas obtenidas")
        return True, resultados, None
        
    except oracledb.DatabaseError as e:
        mensaje_error: str = f"Error de base de datos: {e}"
        print(f"❌ {mensaje_error}")
        return False, None, mensaje_error


def convertir_a_dataframe(
    resultados: list[tuple], 
    cursor: oracledb.Cursor
) -> tuple[pd.DataFrame, str]:
    """Convierte resultados de BD a DataFrame y Markdown.
    
    Args:
        resultados: Lista de tuplas con resultados
        cursor: Cursor con metadata de columnas
        
    Returns:
        Tupla con (DataFrame, texto_markdown)
    """
    # Extraer nombres de columnas
    columnas: list[str] = [col[0] for col in cursor.description]  # type: ignore
    
    # Crear DataFrame
    df: pd.DataFrame = pd.DataFrame(resultados, columns=columnas)
    texto_markdown: str = df.to_markdown(index=False)
    
    return df, texto_markdown


# Ejecutar SQL
exito: bool
resultados: list[tuple] | None
error: str | None
exito, resultados, error = ejecutar_sql_seguro(cursor, sql_generado)

# Solo visualizar si no hubo error
df: pd.DataFrame | None = None
texto_resultado: str | None = None

if exito and resultados is not None:
    df, texto_resultado = convertir_a_dataframe(resultados, cursor)
    print("✅ DataFrame creado correctamente")
    display(df.head())
else:
    print(f"⚠️  No se puede crear DataFrame debido a error: {error}")


✅ Consulta ejecutada: 2 filas obtenidas
✅ DataFrame creado correctamente


,NOMBRE_PRODUCTO,PRECIO_PRODUCTO
0,Teclado mecanico,59.99
1,Mochila ergonomica,45.00


In [18]:
# 🧠 Interpretación con Gemini (solo si hay resultados)

def interpretar_resultados_gemini(
    model: genai.GenerativeModel,
    pregunta: str,
    texto_resultado: str | None
) -> str:
    """Interpreta los resultados usando Gemini.
    
    Args:
        model: Modelo de Gemini configurado
        pregunta: Pregunta original del usuario
        texto_resultado: Resultados en formato Markdown
        
    Returns:
        Interpretación en lenguaje natural
        
    Raises:
        ValueError: Si no hay resultados para interpretar
    """
    # Guard clause
    if not texto_resultado:
        raise ValueError("No hay resultados para interpretar")
    
    prompt: str = f"""
Eres un experto en análisis de datos. Resume e interpreta los resultados de una consulta SQL.

Pregunta original: {pregunta}

Resultados:
{texto_resultado}
"""
    
    response = model.generate_content(prompt)
    interpretacion: str = response.text.strip()
    
    return interpretacion


# Interpretar solo si hubo éxito
if exito and texto_resultado is not None:
    try:
        respuesta_final: str = interpretar_resultados_gemini(model, pregunta, texto_resultado)
        print("✅ Respuesta interpretada por Gemini:")
        print(respuesta_final)
    except ValueError as e:
        print(f"⚠️  {e}")
else:
    print(f"⚠️  No se puede interpretar debido a error en ejecución: {error}")


✅ Respuesta interpretada por Gemini:
Aquí está el resumen e interpretación de los resultados de la consulta SQL sobre las compras de Ana Gamez:

**Resumen:**

Ana Gamez compró dos productos: un teclado mecánico y una mochila ergonómica.

*   El teclado mecánico costó 59.99 unidades monetarias (la moneda no se especifica en los resultados, pero se asume que es la moneda usada en el contexto de la base de datos).
*   La mochila ergonómica costó 45 unidades monetarias.

**Interpretación:**

La consulta revela las preferencias de compra de Ana Gamez.  Podemos inferir lo siguiente:

*   **Enfoque en la Ergonomía:**  La compra de una mochila *ergonómica* sugiere que Ana se preocupa por su comodidad y postura al transportar objetos.
*   **Interés en la Tecnología:** La compra de un *teclado mecánico* indica un posible interés en la tecnología o, alternativamente, que Ana es una persona que escribe mucho y valora la calidad y la sensación táctil de un buen teclado. Podría ser una estudiante, p

## 🧠 Reflexión

- ¿Qué ventajas tiene usar Gemini con API Key?
- ¿Qué limitaciones presenta frente a Vertex AI?
- ¿Cómo podrías extender este flujo para validar o visualizar los resultados?

Este notebook es ideal para prácticas rápidas, pero recuerda que para entornos profesionales, Vertex AI ofrece mayor control y seguridad.

In [ ]:
# 🧹 Limpieza: Cerrar recursos de BD

def cerrar_conexion_segura(cursor: oracledb.Cursor, connection: oracledb.Connection) -> None:
    """Cierra cursor y conexión de Oracle de forma segura.
    
    Args:
        cursor: Cursor a cerrar
        connection: Conexión a cerrar
        
    Note:
        Esta función garantiza el cierre incluso si hay errores
    """
    try:
        if cursor is not None:
            cursor.close()
            print("✅ Cursor cerrado")
    except Exception as e:
        print(f"⚠️  Error al cerrar cursor: {e}")
    
    try:
        if connection is not None:
            connection.close()
            print("✅ Conexión cerrada")
    except Exception as e:
        print(f"⚠️  Error al cerrar conexión: {e}")

# Descomentar para cerrar recursos cuando termines
# cerrar_conexion_segura(cursor, connection)
print("💡 Descomentar la línea anterior para cerrar la conexión al finalizar")
